In [152]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [153]:
import pandas as pd

data = pd.read_csv("C:\\Users\\i00646629\\Documents\\csic_database.csv")
data = data.drop_duplicates()
data = data[:1000]

In [154]:
data['url_split'] = data['URL'].str.split(r"/|:| |=|\?|\&|\.|\+")

In [155]:
data

,Unnamed: 0,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL,url_split
0,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1,"[http, , , localhost, 8080, tienda1, index, js..."
1,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...,"[http, , , localhost, 8080, tienda1, publico, ..."
2,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.j...,"[http, , , localhost, 8080, tienda1, publico, ..."
3,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...,"[http, , , localhost, 8080, tienda1, publico, ..."
4,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...,"[http, , , localhost, 8080, tienda1, publico, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=DE0A136B97BDD065B1A17C2A064403EB,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/miembros/imagene...,"[http, , , localhost, 8080, tienda1, miembros,..."
996,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=0606857038569CED442FCB43F8DEA9A2,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/miembros/imagene...,"[http, , , localhost, 8080, tienda1, miembros,..."
997,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8094BBF79DFDD16C0E5D328EEE570765,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/miembros/index.j...,"[http, , , localhost, 8080, tienda1, miembros,..."
998,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7456D9AD518DDC5916A290D5190F1A11,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/miembros/editar....,"[http, , , localhost, 8080, tienda1, miembros,..."


In [156]:
import re

res_list = []
url_list = list(data['URL'])
for s in url_list:
    tmp = re.split(r"/|:| |=|\?|\&|\.|\+", s)
    res_list.append(list(filter(len, tmp)))

In [157]:
l = []
for i in range(len(res_list)):
    for j in range(len(res_list[i])):
        if res_list[i][j] not in l:
            l.append(res_list[i][j])

In [158]:
encoded_df = pd.DataFrame(columns=l, index=data.index)
#encoded_df['Timestamp'] = data['Timestamp']
for s in l:
    encoded_df.loc[data['URL'].str.contains(s, regex=False), s] = 1

In [159]:
encoded_df = encoded_df.fillna(0)

In [160]:
encoded_df

,http,localhost,8080,tienda1,index,jsp,HTTP,1,publico,anadir,...,Subauste,stadlen%40asociaciondearquitectoslatinoamericanos,tc,60586849L,Rocafort%2C,169,4C,Cerralbo,45569,1918370923327655
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
996,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
997,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
998,1,1,1,1,0,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1


In [161]:
data = np.array(encoded_df)
data

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int64)

In [162]:
timesteps = data.shape[:-1]
n_features = encoded_df.shape[-1]

In [163]:
timesteps

(1000,)

In [164]:
n_features

1036

In [165]:
timesteps = 10
X, y = temporalize(X = data, y = np.zeros(len(data)), lookback = timesteps)
 

X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)
X

array([[[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]],

       ...,

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 

In [166]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 10, 128)           596480    
                                                                 
 lstm_21 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_5 (RepeatVect  (None, 10, 64)           0         
 or)                                                             
                                                                 
 lstm_22 (LSTM)              (None, 10, 64)            33024     
                                                                 
 lstm_23 (LSTM)              (None, 10, 128)           98816     
                                                                 
 time_distributed_5 (TimeDis  (None, 10, 1036)         133644    
 tributed)                                            

In [167]:
model.fit(X, X, epochs=300, batch_size=5, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.001 -0.001 -0.001]
  [ 1.     1.     1.    ...  0.     0.     0.   ]]

 [[ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ... -0.001 -0.001 -0.001]
  [ 1.     1.     1.    ... -0.    -0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]]

 [[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]]

 ...

 [[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 

In [168]:
for l in range (len(yhat)):
    print (np.mean(np.power(yhat[l] - X[l], 4)))

0.00036180290067006163
0.00035583046624207935
6.893265610509457e-05
6.50231865855798e-05
6.640508640029612e-05
6.496484338979998e-05
6.521773193335414e-05
6.50713026382533e-05
6.919669916518725e-05
6.650519749474478e-05
2.0564842802289562e-08
1.3062110034381747e-08
8.47739039699936e-08
1.6220907298235826e-10
9.016520637487696e-10
0.0001705728771206544
0.00016114360095828895
0.00017245426905973592
0.00017324809318977185
0.00017452083771901536
0.0001728059148696502
0.00018159619452476426
0.0001698334231060076
0.00019611556369666757
0.0001878619237111133
1.107579444789782e-09
0.00010097513221926398
0.00010295313211647828
0.00021691691788586537
0.00021601651472919197
0.0002136856935858066
0.00021495299619929092
0.0002192449975507159
0.00021151136003407078
0.00021658908288017494
0.00023294809900808032
0.0002335298960223759
0.00024805123060836
9.2516962684523e-05
8.297288327007025e-05
9.823605706649271e-05
9.474432292958901e-05
9.337525446528949e-05
8.32250231620983e-05
8.841672756008815e-05

In [169]:
mse

0.0009330699753679929

In [170]:
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.001 -0.001 -0.001]
  [ 1.     1.     1.    ...  0.     0.     0.   ]]

 [[ 1.     1.     1.    ...  0.     0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ... -0.001 -0.001 -0.001]
  [ 1.     1.     1.    ... -0.    -0.     0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]]

 [[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  ...
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 1.     1.     1.    ... -0.    -0.    -0.   ]]

 ...

 [[ 1.     1.     1.    ... -0.    -0.    -0.   ]
  [ 